In [1]:
import pandas as pd
from pearlsim.ml_utilities import *
import os
import random
import numpy as np

In [2]:
current_data = pd.read_csv("training_data/current_target.csv", index_col=0).drop(columns=["power"])

In [3]:
file_names = os.listdir("training_data/")
concentrations = []
for name in file_names:
    if "bumat0" in name:
        concentrations += extract_from_bumat("training_data/"+name)


In [4]:
file_name = "burnup_models/burnup0.serpent"

input_s = generate_pebble_burnup_model("pebble_gFHR_template.serpent", current_data.iloc[10], 1120, concentrations[3], 959, 6)

with open(file_name, 'w') as f:
    f.write(input_s)

In [5]:
np.random.seed(1)
iterations = 500
time_steps = [0.06525, 0.2, 1.5, 3.0, 4.6, 6.525, 6.525, 6.525]
conc_array = []
current_array = []
day_array = []
power_array = []



for i in range(1,iterations+1):
    conc = concentrations[np.random.randint(len(concentrations))]
    current = current_data.iloc[np.random.randint(len(current_data))]
    days = time_steps[ np.random.randint(len(time_steps)) ]
    power = np.random.randint(800, high=3000)
    power_array += [power]
    conc_array += [conc]
    current_array += [current]
    day_array += [days]
    file_name = f"burnup_models/burnup{i}.serpent"
    input_s = generate_pebble_burnup_model("pebble_gFHR_template.serpent", current, power, conc, 959,  days)

    #with open(file_name, 'w') as f:
    #    f.write(input_s)

num_cores = 20
iterations = 1000
for i in range(1,iterations+1):

    file_name = f"burnup{i}.serpent"
    os.system(f"sss2_2_0 {file_name} -omp {num_cores}")

In [6]:
features_df = pd.DataFrame({"depletion_time": day_array})
current_df = pd.DataFrame(current_array).reset_index(drop=True)
conc_df = pd.DataFrame(conc_array).fillna(0)
features_df = features_df.join(current_df)
features_df = features_df.join(conc_df)
features_df['power'] = power_array
features_df.iloc[0:5]

,depletion_time,2.9005e-08,9.9e-08,2.1e-07,4.525e-07,7.985000000000001e-07,1.061e-06,1.5025e-06,2.9275e-06,6.9385e-06,...,62147<lib>,62148<lib>,62149<lib>,62150<lib>,62151<lib>,621520,631510,92235<lib>,92238<lib>,power
0,4.600,2.126150e+14,5.195180e+14,5.748160e+14,3.973870e+14,1.264600e+14,2.376000e+13,8.956950e+13,1.416580e+14,1.624290e+14,...,2.135803e-08,3.553372e-08,8.929475e-08,0.000001,2.015638e-07,2.152646e-07,7.349988e-11,0.004016,0.018843,1896
1,3.000,2.301940e+14,5.653970e+14,6.157750e+14,4.178810e+14,1.131750e+14,2.829020e+13,1.121280e+14,1.649230e+14,1.915860e+14,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.004670,0.018973,1760
2,4.600,1.361290e+14,3.365870e+14,3.607000e+14,1.822650e+14,2.672060e+13,1.511560e+12,2.970530e+13,2.870360e+13,5.100380e+13,...,2.226552e-07,5.757140e-07,8.451241e-08,0.000004,2.399743e-07,9.411487e-07,2.997949e-10,0.003077,0.018654,2002
3,6.525,4.433100e+14,9.815350e+14,9.476020e+14,5.650410e+14,1.337090e+14,5.945700e+13,1.086090e+14,1.727270e+14,2.245720e+14,...,4.947674e-08,1.259036e-07,7.752272e-08,0.000003,2.264037e-07,5.364294e-07,9.774726e-11,0.003546,0.018794,1052
4,6.525,2.093670e+14,4.521590e+14,4.341860e+14,2.476010e+14,5.047690e+13,2.374190e+13,5.077140e+13,9.692880e+13,8.705620e+13,...,7.428893e-07,2.328590e-06,9.011014e-08,0.000008,2.453769e-07,1.935729e-06,6.223075e-10,0.002208,0.018457,1362


In [7]:
features_df.to_csv("training_data/burnup_features.csv")

In [5]:
features_df = pd.read_csv("training_data/burnup_features.csv", index_col=0)


FileNotFoundError: [Errno 2] File b'training_data/burnup_features.csv' does not exist: b'training_data/burnup_features.csv'

In [8]:
display(features_df)

,depletion_time,2.9005e-08,9.9e-08,2.1e-07,4.525e-07,7.985000000000001e-07,1.061e-06,1.5025e-06,2.9275e-06,6.9385e-06,...,62147<lib>,62148<lib>,62149<lib>,62150<lib>,62151<lib>,621520,631510,92235<lib>,92238<lib>,power
0,4.60000,2.126150e+14,5.195180e+14,5.748160e+14,3.973870e+14,1.264600e+14,2.376000e+13,8.956950e+13,1.416580e+14,1.624290e+14,...,2.135803e-08,3.553372e-08,8.929475e-08,0.000001,2.015638e-07,2.152646e-07,7.349988e-11,0.004016,0.018843,1896
1,3.00000,2.301940e+14,5.653970e+14,6.157750e+14,4.178810e+14,1.131750e+14,2.829020e+13,1.121280e+14,1.649230e+14,1.915860e+14,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.004670,0.018973,1760
2,4.60000,1.361290e+14,3.365870e+14,3.607000e+14,1.822650e+14,2.672060e+13,1.511560e+12,2.970530e+13,2.870360e+13,5.100380e+13,...,2.226552e-07,5.757140e-07,8.451241e-08,0.000004,2.399743e-07,9.411487e-07,2.997949e-10,0.003077,0.018654,2002
3,6.52500,4.433100e+14,9.815350e+14,9.476020e+14,5.650410e+14,1.337090e+14,5.945700e+13,1.086090e+14,1.727270e+14,2.245720e+14,...,4.947674e-08,1.259036e-07,7.752272e-08,0.000003,2.264037e-07,5.364294e-07,9.774726e-11,0.003546,0.018794,1052
4,6.52500,2.093670e+14,4.521590e+14,4.341860e+14,2.476010e+14,5.047690e+13,2.374190e+13,5.077140e+13,9.692880e+13,8.705620e+13,...,7.428893e-07,2.328590e-06,9.011014e-08,0.000008,2.453769e-07,1.935729e-06,6.223075e-10,0.002208,0.018457,1362
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,6.52500,4.772170e+14,1.150680e+15,1.205580e+15,7.404220e+14,1.762270e+14,5.942860e+13,1.784830e+14,1.973040e+14,2.599170e+14,...,4.619826e-07,1.062539e-06,8.739929e-08,0.000005,2.383095e-07,1.191666e-06,4.580879e-10,0.002839,0.018619,1673
496,6.52500,5.545870e+14,1.405500e+15,1.449500e+15,8.023410e+14,2.072850e+14,7.452540e+13,1.768590e+14,2.358440e+14,3.164100e+14,...,2.182052e-07,4.752298e-07,9.302983e-08,0.000003,2.394241e-07,7.183858e-07,2.833258e-10,0.003346,0.018720,1748
497,3.00000,1.954040e+14,4.519390e+14,4.665500e+14,2.481210e+14,8.189530e+13,2.545170e+13,6.760400e+13,9.423130e+13,1.041840e+14,...,2.981718e-08,5.726169e-08,9.198688e-08,0.000001,2.199621e-07,3.011336e-07,9.456706e-11,0.003870,0.018814,1911
498,0.06525,1.498630e+14,4.384040e+14,4.965960e+14,3.243180e+14,8.831400e+13,3.281720e+13,7.021320e+13,1.525440e+14,1.373560e+14,...,7.592938e-07,1.975854e-06,8.613887e-08,0.000007,2.390738e-07,1.738156e-06,6.168026e-10,0.002358,0.018528,2127


In [9]:
file_names = os.listdir("burnup_models/")
concentration_output = []
for i in range(1,1+iterations):
    try:
        concentration_output += extract_from_bumat(f"burnup_models/burnup{i}.serpent.bumat1")
    except:
        print(f"Could not find burnup_models/burnup{i}.serpent.bumat1")

In [10]:
post_burn_conc_df = pd.DataFrame(concentration_output).fillna(0)

In [11]:
features_df.iloc[0:len(post_burn_conc_df)].to_csv("training_data/burnup_features.csv")
post_burn_conc_df.to_csv("training_data/burnup_target.csv")